In [63]:
import nltk
from scipy.spatial import distance

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Kết nối Colab với Google Drive để lấy dữ liệu

In [20]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Cài đặt thư viện Gensim cho việc train model Word2Vec

In [21]:
!pip install gensim

Import các thư viện cần thiết

In [22]:
import os
import pandas as pd
import string
from gensim.models import Word2Vec

Load tập dữ liệu từ thư mục trong Google Drive

In [54]:
import pandas as pd

PATH = '/content/gdrive/My Drive/1970513_Assignment3/'
dataset = pd.read_csv(PATH + 'bbc_news_train.csv')
data = dataset['Text'].values
print(data[:10])
print(len(data))
print(type(data))

['worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practices used by worldcom. mr eb

In [55]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import re 

def text_cleaner(text):
  for doc in text:
    # lower case text
    text = text.lower()
    text = re.sub(r"'s\b","",text)
    # remove punctuations
    text = re.sub("[^a-zA-Z]", " ", text)
    return text

def preprocess_text(corpus):
    _corpus = corpus.copy()
    all_tokens = []
    for index, doc in enumerate(_corpus):
        clean_doc = text_cleaner(doc)
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(clean_doc)
        doc_tokens = [w for w in word_tokens if not w in stop_words]
        all_tokens += doc_tokens
        _corpus[index] = doc_tokens

    return _corpus, all_tokens

corpus_token, all_tokens =  preprocess_text(data)
print(corpus_token[:10])
print(len(corpus_token))
print(type(corpus_token))

[list(['worldcom', 'ex', 'boss', 'launches', 'defence', 'lawyers', 'defending', 'former', 'worldcom', 'chief', 'bernie', 'ebbers', 'battery', 'fraud', 'charges', 'called', 'company', 'whistleblower', 'first', 'witness', 'cynthia', 'cooper', 'worldcom', 'ex', 'head', 'internal', 'accounting', 'alerted', 'directors', 'irregular', 'accounting', 'practices', 'us', 'telecoms', 'giant', 'warnings', 'led', 'collapse', 'firm', 'following', 'discovery', 'bn', 'bn', 'accounting', 'fraud', 'mr', 'ebbers', 'pleaded', 'guilty', 'charges', 'fraud', 'conspiracy', 'prosecution', 'lawyers', 'argued', 'mr', 'ebbers', 'orchestrated', 'series', 'accounting', 'tricks', 'worldcom', 'ordering', 'employees', 'hide', 'expenses', 'inflate', 'revenues', 'meet', 'wall', 'street', 'earnings', 'estimates', 'ms', 'cooper', 'runs', 'consulting', 'business', 'told', 'jury', 'new', 'york', 'wednesday', 'external', 'auditors', 'arthur', 'andersen', 'approved', 'worldcom', 'accounting', 'early', 'said', 'andersen', 'give

Train model Word2Vec. Trong quá trình train, có bạn có thể chỉnh sử dụng GPU của Colab để đẩy nhanh tiến độ bằng cách vào Rumtime => Change Runtime Type và chọn GPU

In [56]:

# Skipgram model

model = Word2Vec(corpus_token, min_count = 1, size =300, window = 3, sg = 1)
model.wv.save(PATH + 'w2v_result/word2vec_skipgram.bin')

# CBOW model

model = Word2Vec(corpus_token, min_count = 1, size =300, window = 3, sg = 0)
model.wv.save(PATH + 'w2v_result/word2vec_CBOW.bin')



Sau khi có model. Ta tiến hành load mô hình

In [77]:
import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
path = PATH + 'w2v_result/word2vec_CBOW.bin'
path_ = PATH + 'w2v_result/word2vec_skipgram.bin'
model = gensim.models.KeyedVectors.load(path)
model_ = gensim.models.KeyedVectors.load(path_)

Test kết quả

In [58]:
from collections import Counter

word_counts = Counter(all_tokens)
print(word_counts.most_common(25))

[('said', 4839), ('mr', 1974), ('would', 1714), ('year', 1528), ('also', 1426), ('new', 1338), ('people', 1324), ('us', 1264), ('one', 1249), ('could', 1032), ('first', 934), ('last', 893), ('two', 889), ('time', 882), ('world', 805), ('uk', 781), ('government', 752), ('film', 646), ('years', 644), ('best', 642), ('bn', 613), ('make', 613), ('told', 591), ('three', 584), ('made', 580)]


In [59]:
model.most_similar(positive=['said'],topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('kennedy', 0.9955618977546692),
 ('howard', 0.9950141906738281),
 ('flynn', 0.9949874877929688),
 ('makower', 0.9948269128799438),
 ('hockney', 0.9943737983703613),
 ('yeo', 0.9927228093147278),
 ('prescott', 0.9918718338012695),
 ('brown', 0.9915912747383118),
 ('boateng', 0.9904758930206299),
 ('allard', 0.9899778366088867)]

In [60]:
model.similar_by_vector(model['said'], topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('said', 1.0),
 ('kennedy', 0.9955618977546692),
 ('howard', 0.9950141906738281),
 ('flynn', 0.9949874877929688),
 ('makower', 0.9948269128799438),
 ('hockney', 0.9943737983703613),
 ('yeo', 0.9927228093147278),
 ('prescott', 0.9918718338012695),
 ('brown', 0.9915912747383118),
 ('boateng', 0.9904758930206299)]

In [71]:
print(model['said'])
print(model['said'].shape)
print(type(model))

[-3.45946074e-01 -1.65773869e-01 -1.28771633e-01  2.46047884e-01
 -3.07497054e-01 -3.10133845e-01 -1.76596358e-01  3.94541562e-01
 -6.34863153e-02  4.39677611e-02 -2.31068909e-01 -2.11181030e-01
  3.82793128e-01 -2.64688712e-02  1.19091198e-01  1.26822308e-01
 -9.78531957e-01 -1.10494927e-01  5.90011925e-02  5.39726138e-01
 -9.65298787e-02 -3.87886584e-01  1.87405288e-01 -4.86526228e-02
 -1.40208334e-01 -2.09499851e-01  3.07948351e-01  2.30541587e-01
  2.06666872e-01  1.08530581e-01  8.82699639e-02 -6.14335060e-01
  5.93907356e-01  3.78434569e-01 -6.04549468e-01  7.35484064e-02
 -4.17210340e-01 -3.27019781e-01 -7.33903274e-02  1.40057430e-01
 -2.48571411e-02  3.26732218e-01 -7.07638264e-02 -1.58289328e-01
  2.47773275e-01  4.47780818e-01 -2.83680856e-02 -4.86474246e-01
  6.68443203e-01  3.24787498e-01  2.93053240e-01  3.27454329e-01
 -3.73365320e-02 -4.89906073e-01  5.04486978e-01 -4.72580642e-01
 -1.41447084e-02  6.05388880e-02  7.70773217e-02  8.53583813e-02
  8.14420521e-01 -1.98315

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


##CBOW Model

In [79]:
interesting_words, counter = zip(*word_counts.most_common(20))
print(interesting_words)

print("OUTPUT EMBEDDING")
words_eb = []
for w in interesting_words:
  words_eb.append(model[w])
  print(model[w])

print("OUTPUT DISTANCE")
for i, w in enumerate(words_eb):
  i_next = (i+1)%len(words_eb)
  print(str(i) + " vs " + str(i_next)  + ": " + str(distance.euclidean(words_eb[i], words_eb[i_next])))

print("OUTPUT AVG DISTANCE WITH OTHERS")
for i in range(len(words_eb)):
  words_eb_ = list(words_eb.copy())
  words_eb_.pop(i)
  dist_sum = 0
  for k in range(len(words_eb_)):
    dist_sum += distance.euclidean(words_eb[i], words_eb_[k])
  dist_avg = dist_sum/len(words_eb)
  print("The avg distance of " + str(i) + ": " + str(dist_avg))

('said', 'mr', 'would', 'year', 'also', 'new', 'people', 'us', 'one', 'could', 'first', 'last', 'two', 'time', 'world', 'uk', 'government', 'film', 'years', 'best')
OUTPUT EMBEDDING
[-3.45946074e-01 -1.65773869e-01 -1.28771633e-01  2.46047884e-01
 -3.07497054e-01 -3.10133845e-01 -1.76596358e-01  3.94541562e-01
 -6.34863153e-02  4.39677611e-02 -2.31068909e-01 -2.11181030e-01
  3.82793128e-01 -2.64688712e-02  1.19091198e-01  1.26822308e-01
 -9.78531957e-01 -1.10494927e-01  5.90011925e-02  5.39726138e-01
 -9.65298787e-02 -3.87886584e-01  1.87405288e-01 -4.86526228e-02
 -1.40208334e-01 -2.09499851e-01  3.07948351e-01  2.30541587e-01
  2.06666872e-01  1.08530581e-01  8.82699639e-02 -6.14335060e-01
  5.93907356e-01  3.78434569e-01 -6.04549468e-01  7.35484064e-02
 -4.17210340e-01 -3.27019781e-01 -7.33903274e-02  1.40057430e-01
 -2.48571411e-02  3.26732218e-01 -7.07638264e-02 -1.58289328e-01
  2.47773275e-01  4.47780818e-01 -2.83680856e-02 -4.86474246e-01
  6.68443203e-01  3.24787498e-01  2.93

- Khoảng cách giữa các từ khác nhau trong w2v cbow thể hiện được sự khác biệt giữa các từ hơn so với auto encoder
- Khoảng cách trung bình giữa của 1 từ so với các từ khác có sự khác biệt rõ rệt, ví dụ như giữa khoảng cách trung bình của từ 0 với từ 4
- Những từ khác nhau cho ra vector khác nhau

##SKIP-GRAM Model

In [80]:
print("OUTPUT EMBEDDING")
words_eb = []
for w in interesting_words:
  words_eb.append(model_[w])
  print(model_[w])

print("OUTPUT DISTANCE")
for i, w in enumerate(words_eb):
  i_next = (i+1)%len(words_eb)
  print(str(i) + " vs " + str(i_next)  + ": " + str(distance.euclidean(words_eb[i], words_eb[i_next])))

print("OUTPUT AVG DISTANCE WITH OTHERS")
for i in range(len(words_eb)):
  words_eb_ = list(words_eb.copy())
  words_eb_.pop(i)
  dist_sum = 0
  for k in range(len(words_eb_)):
    dist_sum += distance.euclidean(words_eb[i], words_eb_[k])
  dist_avg = dist_sum/len(words_eb)
  print("The avg distance of " + str(i) + ": " + str(dist_avg))

OUTPUT EMBEDDING
[-0.03666767 -0.07749122  0.01912094  0.04617509 -0.18109867 -0.03727398
 -0.20746809  0.11963679  0.01337447  0.14624704 -0.05865474  0.03068375
  0.19797805 -0.02446565  0.15594947  0.1331531  -0.28551888 -0.00930997
  0.00426076  0.1604041   0.13693637 -0.09387046  0.15427099 -0.00958226
 -0.06769436  0.08786194  0.11289887  0.08228429  0.08126306  0.10245482
 -0.11124861 -0.22477183  0.25831285  0.0987855  -0.18197845  0.10240442
 -0.13775644 -0.1536123  -0.08441017  0.05591503  0.07233318  0.11216524
 -0.07124855 -0.11548794 -0.02448913  0.17157324 -0.0254758  -0.18260655
  0.28706774  0.0507797   0.02329796  0.08201615 -0.0108915  -0.23659928
  0.14749214 -0.2326768   0.01570402  0.07960038  0.18912008 -0.06236877
  0.28749418  0.01012769  0.30038285  0.20048909  0.06542203  0.06681392
  0.00203945 -0.0273636   0.09916957 -0.10909747  0.07251805  0.13517998
  0.05486294 -0.09830343  0.00622215 -0.14063367 -0.21262369  0.04970004
 -0.03418674  0.0113241  -0.009471

- Trung bình khoảng cách giữa 1 từ so các từ khác cũng không có sự khác biệt rõ rệt như cbow